# AgentTuning
## [AgentTuning](https://thudm.github.io/AgentTuning/) is an attempt to instruction-tune LLMs using interaction trajectories across multiple agent tasks. Evaluation results indicate that AgentTuning enables the agent capabilities of LLMs with robust generalization on unseen agent tasks while remaining good on general language abilities. 

## In this cookbook, We will show how to use the open-sourced **AgentInstruct** dataset to finetune Anyscale Endpoints

In [ ]:
!pip install pandas pyarrow s3fs fastparquet

### 1. Download AgentInstruct dataset from [Hugging Face](https://huggingface.co/datasets/THUDM/AgentInstruct), and convert the PARQUET file into JSONL format 

In [39]:
### Parquet to GPT-format jsonl
import pandas as pd
import json
filenames = ['alfworld-00000-of-00001-302ad687bb3817a4.parquet',
             'webshop-00000-of-00001-9f2ae60445e11b4e.parquet',
             'mind2web-00000-of-00001-fc25d47330eea0fc.parquet',
             'kg-00000-of-00001-9e159f6d0557d229.parquet',
             'os-00000-of-00001-971539c34fcc7500.parquet',
             'db-00000-of-00001-916a87c4725da8c0.parquet']       
             
def parquet2jsonl(filename, outfn):
    df = pd.read_parquet(filename)
    outfile =  open(outfn, 'w')
    entry = {}
    for row in df['conversations']:
        entry['messages']=[]
        for msg in row:
            if msg['from'] == 'human':
                entry['messages'].append({'role':'user','content':msg['value']})
            elif msg['from'] == 'gpt':
                entry['messages'].append({'role':'assistant','content':msg['value']})
        json.dump(entry, outfile)
        outfile.write('\n')

for fn in filenames:
    filename = '/AGNETINSTRUCT_FOLDER/'+fn
    outfn = fn.split('-')[0]+".jsonl"
    parquet2jsonl(filename, outfn)

In [62]:
### Combine into one whole jsonl file
infiles=['alfworld.jsonl','webshop.jsonl','mind2web.jsonl','kg.jsonl','os.jsonl','db.jsonl']
outfile =  open('agentinstruct_all.jsonl', 'w')
for ifile in infiles:
    infile = open(ifile,'r')
    for row in infile:
        entry = json.loads(row)
        json.dump(entry, outfile)
        outfile.write('\n')

In [65]:
## split into train and val datasets
DATA_PATH = "agentinstruct_all.jsonl"
# Load the dataset
with open(DATA_PATH, 'r', encoding='utf-8') as f:
    items = [json.loads(line) for line in f]
with open('agentinstruct_train.jsonl', 'w') as outfile:
    for i, entry in enumerate(items):
        if i%5:
            json.dump(entry, outfile)
            outfile.write('\n')
with open('agentinstruct_val.jsonl', 'w') as outfile:
    for i, entry in enumerate(items):
        if i%5==0:
            json.dump(entry, outfile)
            outfile.write('\n')

### 2. Data Validation check

In [ ]:
from finetune_utils import finetune_check
finetune_check('./agentinstruct_train.jsonl')

## 3. Finetune with training and val datasets

In [93]:
from finetune_utils import finetune_run
finetune_run('./agentinstruct_train.jsonl','./agentinstruct_val.jsonl',
             token='ANYSCALE_API_TOKEN', model='meta-llama/Llama-2-13b-chat-hf', suffix='agentinstruct')

<OpenAIObject id=eftjob_g7fumcde5n1tcbrwfwz2brlal4 at 0x7fd763f928b0> JSON: {
  "result_files": [],
  "trained_tokens": null,
  "hyperparameters": {
    "n_epochs": null,
    "context_length": null
  },
  "training_file": "file_67mtyt94frq7qtzcyirplyvlnf",
  "validation_file": "file_8fstlskmnmbhsymsl1g4ava7zj",
  "model": "meta-llama/Llama-2-13b-chat-hf",
  "id": "eftjob_g7fumcde5n1tcbrwfwz2brlal4",
  "created_at": "2023-10-24T01:41:15.123886+00:00",
  "finished_at": null,
  "fine_tuned_model": "meta-llama/Llama-2-13b-chat-hf:agentinstruct:8VRVuzi",
  "status": "pending",
  "error": null,
  "creator_id": "user_ymu3y6unv8k55r54ifgsdk533j"
}

In [96]:
import openai
## Check funetune status
openai.FineTuningJob.retrieve("eftjob_g7fumcde5n1tcbrwfwz2brlal4")

<FineTuningJob id=eftjob_g7fumcde5n1tcbrwfwz2brlal4 at 0x7fd1228d3e00> JSON: {
  "result_files": [
    "file_8vpyqr9x6iwlxzsvhha53jk4ta"
  ],
  "trained_tokens": null,
  "hyperparameters": {
    "n_epochs": null,
    "context_length": null
  },
  "training_file": "file_67mtyt94frq7qtzcyirplyvlnf",
  "validation_file": "file_8fstlskmnmbhsymsl1g4ava7zj",
  "model": "meta-llama/Llama-2-13b-chat-hf",
  "id": "eftjob_g7fumcde5n1tcbrwfwz2brlal4",
  "created_at": "2023-10-24T01:41:15.123886+00:00",
  "finished_at": null,
  "fine_tuned_model": "meta-llama/Llama-2-13b-chat-hf:agentinstruct:8VRVuzi",
  "status": "running",
  "error": null,
  "creator_id": "user_ymu3y6unv8k55r54ifgsdk533j"
}

### 4. Run inference on finetuned models

In [6]:
import openai
content="your query here"
OPENAI_API_BASE="https://console.endpoints.anyscale.com/m/v1"
OPENAI_API_KEY="ANYSCALE_API_TOKEN"
chat_completion = openai.ChatCompletion.create(
    api_base=OPENAI_API_BASE,
    api_key=OPENAI_API_KEY,
    model="meta-llama/Llama-2-13b-chat-hf:agentinstruct:8VRVuzi",
    messages=[{"role": "user", "content": content}],
    temperature=0
)
print(chat_completion)

{
  "id": "meta-llama/Llama-2-13b-chat-hf:agentinstruct:8VRVuzi-2ce2cd09a92398cddd31fea685d13fe6",
  "object": "text_completion",
  "created": 1698196096,
  "model": "meta-llama/Llama-2-13b-chat-hf:agentinstruct:8VRVuzi",
  "choices": [
    {
      "message": {
        "role": "assistant",
        "content": " I'll be happy to help with your query. Please provide the query you'd like me to execute. "
      },
      "index": 0,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 11,
    "completion_tokens": 25,
    "total_tokens": 36
  }
}
